In [1]:
import numpy as np
import pandas as pd
import mne
import yasa
from scipy import signal

C:\Users\Origin\AppData\Roaming\Python\Python38\site-packages\antropy\fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
def Filter_init(l_freq_p=0.5, h_freq_p=4, l_freq_s=0.1, h_freq_s=5, fs=500, order=2):
    LowPassFre = h_freq_p
    HighPassFre = l_freq_p

    h_freq_p = h_freq_p * 2 / fs
    l_freq_p = l_freq_p * 2 / fs
    wp = [l_freq_p, h_freq_p]

    h_freq_s = h_freq_s * 2 / fs
    l_freq_s = l_freq_s * 2 / fs
    ws = [l_freq_s, h_freq_s]

    N, wn = signal.buttord(wp, ws, 5, 40)
    N = order

    b, a = signal.butter(N, wn, "bandpass")

    w, gd = signal.group_delay((b, a), w=500, fs=500)

    Filter_Delay = np.mean(gd[np.min(np.where(w > HighPassFre)): np.max(np.where(w < LowPassFre))])

    Filter_Delay = round(Filter_Delay)
    
    Filter_Delay = Filter_Delay // 2

    delay = np.zeros(Filter_Delay, dtype=np.float64)
    

    return b, a, N, Filter_Delay, delay

b, a, N, Filter_Delay, delay = Filter_init()

C:\Users\Origin\AppData\Local\Temp\ipykernel_44892\2460382674.py:18: UserWarning: The filter's denominator is extremely small at frequencies [0.000],             around which a singularity may be present
  w, gd = signal.group_delay((b, a), w=500, fs=500)


In [3]:
def SO_filter_single(b, a, x):
    x = np.concatenate([x, delay])
    y = signal.lfilter(b, a, x)
    y = y[Filter_Delay:]
    return y

In [4]:
SaveLabel = "F:/35-2023_10_19-HongJiakang-Female"
dataPath = SaveLabel + "/Sleep.vhdr"

In [5]:
raw = mne.io.read_raw_brainvision(dataPath, preload=True)
raw.pick(["C3", "M2"])
raw.set_eeg_reference(["M2"])
raw.pick(["C3"])
raw.filter(0.1, 40)
raw._data[0] = raw._data[0] * -1

Extracting parameters from F:/35-2023_10_19-HongJiakang-Female/Sleep.vhdr...
Setting channel info structure...
Reading 0 ... 13831879  =      0.000 ... 27663.758 secs...


C:\Users\Origin\AppData\Roaming\Python\Python38\site-packages\outdated\utils.py:14: OutdatedCheckFailedWarning: Failed to check for latest version of package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 16501 samples (33.002 sec)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [6]:
real_annotations = raw.annotations
real_annotations = pd.DataFrame(real_annotations)
real_onset = real_annotations["onset"]
real_duration = real_annotations["duration"]
real_description = real_annotations["description"]

so_onset = []
so_duration = []
so_description = []

for i in range(len(real_description)):
    if "50" in real_description[i]:
        so_onset.append(real_onset[i])
        so_duration.append(0.02)
        so_description.append("SO_real")

In [7]:
annotations_SO_real = mne.Annotations(onset=so_onset, duration=so_duration, description=so_description)
raw.set_annotations(annotations_SO_real)
mne.export.export_raw(SaveLabel + "/so_real.edf", raw, fmt="edf")

C:\Users\Liu Lab\AppData\Local\Temp\ipykernel_12988\1597458532.py:3: RuntimeWarning: EDF format requires equal-length data blocks, so 0.24 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(SaveLabel + "/so_real.edf", raw, fmt="edf")


In [8]:
SO_Label = SaveLabel + "/Annotations_SO.csv"

annotations_SO = pd.read_csv(SO_Label)
annotations_so = mne.Annotations(onset=list(annotations_SO["onset"]), duration=list(annotations_SO["duration"]), description=list(annotations_SO["description"]))

raw.set_annotations(annotations_so)
mne.export.export_raw(SaveLabel + "/so_simu.edf", raw, fmt="edf")

C:\Users\Liu Lab\AppData\Local\Temp\ipykernel_12988\1869503004.py:7: RuntimeWarning: EDF format requires equal-length data blocks, so 0.24 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(SaveLabel + "/so_simu.edf", raw, fmt="edf")


In [7]:
sw = yasa.sw_detect(data=raw, sf=500, ch_names="C3", freq_sw=[0.5, 4], amp_neg=[50, 200], amp_pos=[15, 150])
slow_Wave_offline = sw.summary()

offline_description = []
for i in range(len(slow_Wave_offline["Start"])):
    offline_description.append("SO_yasa")
offline_annotations = {"onset": slow_Wave_offline["Start"], "duration": slow_Wave_offline["Duration"], "description": offline_description}
offline_annotations = pd.DataFrame(offline_annotations)
annotations_so_offline = mne.Annotations(onset=list(offline_annotations["onset"]), duration=list(offline_annotations["duration"]), description=list(offline_annotations["description"]))

raw.set_annotations(annotations_so_offline)
mne.export.export_raw(SaveLabel + "/so_offline.edf", raw, fmt="edf")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
C:\Users\Origin\AppData\Local\Temp\ipykernel_44892\3277078547.py:12: RuntimeWarning: EDF format requires equal-length data blocks, so 0.24 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(SaveLabel + "/so_offline.edf", raw, fmt="edf")


In [10]:
raw.set_annotations(None)

data, _ = raw[:]
data = data[0]
data = signal.filtfilt(b, a, data)

raw._data[0] = data

mne.export.export_raw(SaveLabel + "/so_filt.edf", raw, fmt="edf")

C:\Users\Liu Lab\AppData\Local\Temp\ipykernel_12988\66507233.py:9: RuntimeWarning: EDF format requires equal-length data blocks, so 0.24 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(SaveLabel + "/so_filt.edf", raw, fmt="edf")
